In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,losses,optimizers,datasets

# 10. 卷积神经网络

## 10.1 全连接网络的问题
首先我们来分析全连接网络存在的问题。考虑一个简单的4层全连接层网络，如`图10.1`所示。

<img src="images/10_01.png" style="width:600px;"/>

通过`TensorFlow`快速地搭建此网络模型：

In [2]:
# 创建4层全连接网络
model = keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10),
])

# build模型，并打印模型信息
model.build(input_shape=(4, 784))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  65792     
_________________________________________________________________
dense_2 (Dense)              multiple                  65792     
_________________________________________________________________
dense_3 (Dense)              multiple                  2570      
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


`summary()`函数打印出模型每一层的参数量统计结果。

网络的参数量是怎么计算的呢？对于每一条连接线的权值标量，视作一个参数，因此对输入节点数为$n$，输出节点数为$m$的全连接层来说，$W$张量包含的参数量共有$n \cdot m$个，$b$向量包含的参数量有$m$个，则全连接层的总参数量为$n\cdot m+m$。上面的四层网络总参数量约34万个。如果将单个权值保存为`float`类型的变量，至少需要占用4个字节内存(`Python`语言中`float`占用内存更多)，那么34万个网络参数至少需要约1.34MB内存。网络的训练过程中还需要缓存计算图模型、梯度信息、输入和中间计算结果等，其中梯度相关运算占用资源非常多。

那么训练这样一个网络到底需要多少内存呢？我们可以在现代GPU 备上简单模拟一下资源消耗情况。在`TensorFlow`中，如果不设置显存占用方式，那么默认会占用全部显存。这里将`TensorFlow`的显存使用方式设置为按需分配：

```python
# 获取所有 GPU 设备列表
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 设置 GPU 显存占用为按需分配，增长式
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # 异常处理
        print(e)
```

设置`TensorFlow`按需申请显存资源，这样`TensorFlow`占用的显存大小即为运算需要的数量。在`Batch Size`设置为32的情况下，我们观察到显存占用了约708MB，内存占用约870MB。

### 10.1.1 局部相关性
接下来我们探索如何避免全连接网络的参数量过大的缺陷。为了便于讨论，我们以图片类型数据为输入的场景为例。对于2D的图片数据，在进入全连接层之前，需要将矩阵数据打平成1D向量，然后每个像素点与每个输出节点两两相连，如`图10.2`所示。

<img src="images/10_02.png" style="width:500px;"/>

网络层的每个输出节点与所有的输入节点相连接，用于提取所有输入节点的特征信息，这种稠密的连接方式是全连接层参数量大、计算代价高的根本原因。全连接层也称为`稠密连接层`(Dense Layer)，输出与输入的关系为：
+ $o_j = \sigma\bigg(\displaystyle\sum_{i \in nodes(I)}w_{ij}x_i+b_j\bigg)$

其中$nodes(I)$表示$I$层的节点集合。

那么，输出节点是否有必要和全部的输入节点相连接呢？我们可以分析输入节点对输出节点的重要性分布，仅考虑较重要的一部分输入节点， 而抛弃重要性较低的部分节点，这样输出节点只需要与部分输入节点相连接，表达为：
+ $o_j = \sigma\bigg(\displaystyle\sum_{i \in top(I,j,k)}w_{ij}x_i+b_j\bigg)$

其中$top(I,j,k)$表示$I$层中对于$J$层中的$j$号节点重要性最高的前$k$个节点集合。通过这种方式，可以把全连接层的$\lVert I\rVert\cdot\lVert J\rVert$个权值连接减少到$k\cdot\lVert J\rVert$个，其中$\lVert I\rVert$、$\lVert J\rVert$分布表示$I,J$层的节点数量。

现在问题就转变为探索$I$层输入节点对于$j$号输出节点的重要性分布。找出每个中间节点的重要性分布是件非常困难的事情，我们可以针对于具体问题，利用先验知识把这个问题进一步简化。

在现实生活中，存在着大量以位置或距离作为重要性分布衡量标准的数据，比如股票的走势预测应该更加关注近段时间的数据趋势(时间相关)、图片每个像素点和周边像素点的关联度更大(位置相关)。

以2D图片数据为例，如果简单地认为与当前像素欧式距离(Euclidean Distance)小于和等于$\displaystyle\frac{k}{\sqrt{2}}$的像素点重要性较高，反之则重要性较低，那么我们就很轻松地简化了每个像素点的重要性分布问题。如`图10.3`所示

<img src="images/10_03.png" style="width:300px;"/>

这个高宽为$k$的窗口称为`感受野`(Receptive Field)，它表征了每个像素对于中心像素的重要性分布情况，网格内的像素才会被考虑。

这种基于距离的重要性分布假设特性称为`局部相关性`。在这种重要性分布假设下，全连接层的连接模式变成了如`图10.4`：

<img src="images/10_04.png" style="width:300px;"/>

利用局部相关性的思想，我们把感受野窗口的高、宽记为$k$(实际上感受野的高、宽可以不相等)，当前位置的节点与大小为$k$的窗口内的所有像素相连接，网络层的输入输出关系表达如下：
+ $o_j = \sigma\bigg(\displaystyle\sum_{\text{dist}(i,j) \le \frac{k}{\sqrt{2}}}w_{ij}x_i+b_j\bigg)$

### 10.1.2 权值共享
每个输出节点仅与感受野区域内$k\times k$个输入节点相连接，输出层节点数为$\lVert J\rVert$，则当前层的参数量为$k \times k \times \lVert J\rVert$，相对于全连接层的$\lVert I\rVert\times\lVert J\rVert$，考虑到$k$(如1、3、5)，网络模型参数量减少了很多。

能否再将参数量进一步减少，比如只需要$k\times k$个参数即可完成当前层的计算？答案是肯定的，通过`权值共享`的思想，对于每个输出节点$o_j$，均使用相同的权值矩阵$W$，那么无论输出节点的数量$\lVert J\rVert$是多少，网络层的参数量总是$k\times k$。如`图10.5`所示，在计算左上角 位置的输出像素时，使用权值矩阵：
+ $W = \begin{bmatrix}w_{11}&w_{12}&w_{13}\\w_{21}&w_{22}&w_{23}\\w_{31}&w_{32}&w_{33}\\ \end{bmatrix}$

与对应感受野内部的像素相乘累加，作为左上角像素的输出值；在计算右下方感受野区域时，使用相同的权值参数$W$相乘累加，得到右下角像素的输出值，此时网络层的参数量只有$3 \times 3 = 9$个，且与输入、输出节点数无关。

<img src="images/10_05.png" style="width:300px;"/>

通过运用`局部相关性`和`权值共享`的思想，我们把网络的参数量减少到$k \times k$(准确地说，是在单输入通道、单卷积核的条件下)。这种共享权值的`局部连接层`网络其实就是卷积神经网络。

## 10.2 卷积神经网络
卷积神经网络通过充分利用局部相关性和权值共享的思想，大大地减少了网络的参数量，从而提高训练效率，更容易实现超大规模的深层网络。

本节介绍卷积神经网络层的具体计算流程。以2D图片数据为例，卷积层接受高、宽分别为$h、w$，通道数为$c_{in}$的输入特征图$X$，在$c_{out}$个高、宽都为$k$，通道数为$c_{in}$的 卷积核作用下，生成高、宽分别为$h'、w'$，通道数为$c_{out}$的特征图输出。需要注意的是，卷积核的高宽可以不等。

我们首先从单通道输入、单卷积核的情况开始讨论，然后推广至多通道输入、单卷积核，最后讨论最常用，也是最复杂的多通道输入、多个卷积核的卷积层实现。

### 10.2.1 单通道输入和单卷积核
首先讨论单通道输入$c_{in} = 1$，如灰度图片只有灰度值一个通道，单个卷积核$c_{out} = 1$的情况。以输入$X$为$5 \times 5$的矩阵，卷积核为$3 \times 3$的矩阵为例，如`图10.12`所示。

<img src="images/10_12.png" style="width:500px;"/>

首先移动至输入$X$最左上方(上方的绿色方框)，选中输入$X$上$3 \times 3$的感受野元素，与卷积核(图片中间$3 \times 3$方框)对应元素相乘：
+ $\begin{bmatrix}1&-1&0\\-1&-2&2\\1&2&-2\\ \end{bmatrix} \odot \begin{bmatrix}-1&1&2\\1&-1&3\\0&-1&-2\\ \end{bmatrix} = \begin{bmatrix}-1&-1&0\\-1&2&6\\0&-2&4\\ \end{bmatrix}$

$\odot$表示哈达马积(Hadamard Product)，即矩阵的对应元素相乘，它与矩阵相乘符号@是矩阵的二种最为常见的运算形式。运算后得到$3\times 3$的矩阵，这9个数值全部相加：$−1−1+0−1+2+6+0−2+4=7$，得到标量7，写入输出矩阵第一行、第一列的位置。

完成第一个感受野区域的特征提取后，感受野窗口向右移动一个步长单位(Strides，默认为1)，选中`图10.13`中绿色方框标示的感受野元素，按照同样的计算方法得到输出10，写入第一行、第二列位置。

<img src="images/10_13.png" style="width:500px;"/>

感受野窗口再次向右移动一个步长单位计算输出，如`图10.14`所示。

<img src="images/10_14.png" style="width:500px;"/>

此时感受野已经移动至输入$X$的有效像素的最右边，此时，感受野窗口向下移动一个步长单位并回到当前行的行首位置，继续选中新的感受野元素区域，如`图10.15`所示。

<img src="images/10_15.png" style="width:500px;"/>

按照上述方法，每次感受野向右移动1个步长单位，若超出输入边界，则向下移动1个步长单位，并回到行首，直到感受野移动至最右边、最下方位置，如下`图10.16`所示。

<img src="images/10_16.png" style="width:500px;"/>

### 10.2.2 多通道输入和单卷积核
多通道输入的卷积层更为常见，比如彩色的图片包含了`R/G/B`三个通道，每个通道上面的像素值表示`R/G/B`色彩的强度。下面我们以3通道输入、单个卷积核为例，将单通道输入的卷积运算方法推广到多通道的情况。

如`图10.17`中所示，每行的最左边$5\times5$的矩阵表示输入$X$的`1~3`通道，第2列的$3\times3$矩阵分别表示卷积核的`1~3`通道，第3列的矩阵表示当前通道上运算结果的中间矩阵，最右边一个矩阵表示卷积层运算的最终输出。

<img src="images/10_17.png" style="width:500px;"/>

在多通道输入的情况下，卷积核的通道数需要和输入$X$的通道数量相匹配，卷积核的第$i$个通道和$X$的第$i$个通道运算，得到第$i$个中间矩阵，此时可以视为单通道输入与单卷积核的情况，所有通道的中间矩阵对应元素再次相加，作为最终输出。

具体的计算流程如下：在初始状态，如`图10.17`所示，每个通道上面的感受野窗口同步落在对应通道上面的最左边、最上方位置，每个通道上感受野区域元素与卷积核对应通道上面的矩阵相乘累加，分别得到三个通道上面的输出`7、-11、-1`的中间变量，这些中间变量相加得到输出-5，写入对应位置。

随后，感受野窗口同步在$X$的每个通道上向右移动1个步长单位，如`图10.18`所示，每个通道上面的感受野与卷积核对应通道上面的矩阵相乘累加，得到中间变量`10、20、20`，全部相加得到输出50，写入第一行、第二列元素位置。

<img src="images/10_18.png" style="width:500px;"/>

以此方式同步移动感受野窗口，直至最右边、最下方位置，此时全部完成输入和卷积核的卷积运算，得到$3\times3$的输出矩阵，如`图10.19`所示。

<img src="images/10_19.png" style="width:500px;"/>

整个计算如`图10.20`所示，输入的每个通道处的感受野均与卷积核的对应通道相乘累加，得到与通道数量相等的中间变量，这些中间变量全部相加即得到当前位置的输出值。输入通道的通道数量决定了卷积核的通道数。一个卷积核只能得到一个输出矩阵，无论输入$X$的通道数量。

<img src="images/10_20.png" style="width:300px;"/>

一般来说，一个卷积核只能完成某种逻辑的特征提取，当需要同时提取多种逻辑特征时，可以通过增加多个卷积核来得到多种特征，提高神经网络的表达能力，这就是多通道输入、多卷积核的情况。

### 10.2.3 多通道输入、多卷积核
多通道输入、多卷积核是卷积神经网络中最为常见的形式。

当出现多卷积核时，第$i$($i \in n$，n为卷积核个数)个卷积核与输入$X$运算得到第$i$个输出矩阵(也称为输出张量$O$的通道$i$)，最后全部的输出矩阵在通道维度上进行拼接(Stack操作，创建输出通道数的新维度)，产生输出张量$O$，$O$包含了$n$个通道数。

以3通道输入、2个卷积核的卷积层为例。第一个卷积核与输入$X$运算得到输出$O$的第一个通道，第二个卷积核与输入$X$运算得到输出$O$的第二个通道，如`图10.21`所示，输出的两个通道拼接在一起形成了最终输出$O$。每个卷积核的大小$k$、步长$s$、填充设定等都是统一设置，这样才能保证输出的每个通道大小一致，从而满足拼接的条件。

<img src="images/10_21.png" style="width:500px;"/>

### 10.2.4 步长
在卷积运算中，如何控制感受野布置的密度呢？对于信息密度较大的输入，如物体数 量很多的图片，为了尽可能的少漏掉有用信息，在网络设计的时候希望能够较密集地布置 感受野窗口；对于信息密度较小的输入，比如全是海洋的图片，可以适量的减少感受野窗 口的数量。感受野密度的控制手段一般是通过移动步长(Strides)实现的。

步长是指感受野窗口每次移动的长度单位，对于 2D 输入来说，分为沿𝑥(向右)方向和 𝑦(向下)方向的移动长度。为了简化讨论，这里只考虑𝑥/𝑦方向移动步长相同的情况，这也 是神经网络中最常见的设定。如下图 10.22 所示，绿色实线代表的感受野窗口的位置是当 前位置，绿色虚线代表是上一次感受野所在位置，从上一次位置移动到当前位置的移动长 度即是步长的定义。图 10.22 中感受野沿𝑥 方向的步长为 2，表达为步长𝑠 = 2。